In [1]:
# To calculate Speed and Azimuth of a mid point Midpoint (Lon, Lat) in excel.
print("Hey Nishant")
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os

# Convert decimal degrees to DMS format
def decimal_to_dms(dd, is_lat=True):
    direction = ''
    if is_lat:
        direction = 'N' if dd >= 0 else 'S'
    else:
        direction = 'E' if dd >= 0 else 'W'
    dd = abs(dd)
    degrees = int(dd)
    minutes = int((dd - degrees) * 60)
    seconds = round((dd - degrees - minutes / 60) * 3600)
    return degrees, minutes, seconds, direction

# Get result from GAGE calculator for a lat/lon pair
def get_motion_result(lat_dd, lon_dd):
    lat_d, lat_m, lat_s, lat_dir = decimal_to_dms(lat_dd, is_lat=True)
    lon_d, lon_m, lon_s, lon_dir = decimal_to_dms(lon_dd, is_lat=False)

    data = {
        "lat": lat_d,
        "lat_m": lat_m,
        "lat_s": lat_s,
        "lon": lon_d,
        "lon_m": lon_m,
        "lon_s": lon_s,
        "model": "itrf2020",
        "reference": "NNR",
        "format": "html",
        "submit": "Submit"
    }

    response = requests.post("https://www.unavco.org/software/geodetic-utilities/plate-motion-calculator/plate-motion/model", data=data)
    soup = BeautifulSoup(response.text, "html.parser")
    table = soup.find("table")
    if not table:
        return None

    rows = table.find_all("tr")
    if len(rows) < 2:
        return None

    values = [td.text.strip().replace('\xa0', '') for td in rows[1].find_all("td")]

    return {
        "Latitude (Decimal)": lat_dd,
        "Longitude (Decimal)": lon_dd,
        "Latitude (DMS)": f"{lat_d}° {lat_m}' {lat_s}\" {lat_dir}",
        "Longitude (DMS)": f"{lon_d}° {lon_m}' {lon_s}\" {lon_dir}",
        "Speed (mm/yr)": float(values[3]),
        "Azimuth (°)": float(values[4].replace("°", "")),
        "North Velocity (mm/yr)": float(values[5]),
        "East Velocity (mm/yr)": float(values[6]),
        "Plate": values[7]
    }

# === CONFIGURATION ===
input_file = r"C:\Users\mainw\Downloads\ridges-midpoint.xlsx"
output_folder = r"C:\Users\mainw\Downloads"
output_file = os.path.join(output_folder, "GAGE_Output.xlsx")

# === PROCESSING ===
df_input = pd.read_excel(input_file)

results = []
for index, row in df_input.iterrows():
    try:
        lon_str, lat_str = row["Midpoint (Lon, Lat)"].strip("()").split(",")
        lon = float(lon_str)
        lat = float(lat_str)
        print(f"📍 Processing Row {index + 1}: Latitude = {lat:.6f}, Longitude = {lon:.6f} ...")
        result = get_motion_result(lat, lon)
        if result:
            results.append(result)
            print(f"✅ Success: Speed = {result['Speed (mm/yr)']} mm/yr, Azimuth = {result['Azimuth (°)']}°\n")
        else:
            print(f"⚠️ No result returned from GAGE for row {index + 1}\n")
    except Exception as e:
        print(f"❌ Error processing row {index + 1}: {e}\n")

# Save output
df_output = pd.DataFrame(results)
df_output.to_excel(output_file, index=False)
print(f"✅ All done! Results saved to:\n{output_file}")

Hey Nishant
📍 Processing Row 1: Latitude = -16.431723, Longitude = 67.151206 ...
✅ Success: Speed = 50.52 mm/yr, Azimuth = 50.35°

📍 Processing Row 2: Latitude = -15.719582, Longitude = 67.437041 ...
✅ Success: Speed = 50.84 mm/yr, Azimuth = 50.55°

📍 Processing Row 3: Latitude = 6.033978, Longitude = 68.439295 ...
✅ Success: Speed = 53.94 mm/yr, Azimuth = 52.89°

📍 Processing Row 4: Latitude = -13.087517, Longitude = 66.650260 ...
✅ Success: Speed = 51.52 mm/yr, Azimuth = 51.44°

📍 Processing Row 5: Latitude = -13.482451, Longitude = 66.662688 ...
✅ Success: Speed = 51.39 mm/yr, Azimuth = 51.32°

📍 Processing Row 6: Latitude = -11.083771, Longitude = 66.276616 ...
✅ Success: Speed = 52.06 mm/yr, Azimuth = 52.04°

📍 Processing Row 7: Latitude = -11.451832, Longitude = 66.405707 ...
✅ Success: Speed = 51.98 mm/yr, Azimuth = 51.93°

📍 Processing Row 8: Latitude = -9.448784, Longitude = 67.010870 ...
✅ Success: Speed = 52.73 mm/yr, Azimuth = 52.35°

📍 Processing Row 9: Latitude = -8.46480